In [1]:
import sys
sys.path.insert(0, '/Users/jmmoon/Documents/GitHub/LDPUts')
import gc
from discretizer import discretizer
from client import client
import torch
from server import server_ell2, server_multinomial_genrr, server_multinomial_bitflip, server_multinomial_bitflip_old
from data_generator import data_generator
from discretizer import discretizer
import time
import numpy as np
from scipy.stats import chi2
from utils import chi_sq_dist


device = torch.device("mps")
print(device)

sample_size = 10000
privacy_level = 2.0




n_test = 300
n_permutation = 2
significance_level = 0.05
alphabet_size = 4

p1 = torch.ones(alphabet_size).div(alphabet_size)

bump_size = 0.01
p2 = p1.add(
    torch.remainder(
    torch.tensor(range(alphabet_size)),
    2
    ).add(-1/2).mul(2).mul(bump_size)
)
print(p2)


alphabet_size = 4
    
data_gen = data_generator()
LDPclient = client()


server_bitflip = server_multinomial_bitflip(privacy_level)
server_bitflip_old = server_multinomial_bitflip_old(privacy_level)


    

mps
tensor([0.2400, 0.2600, 0.2400, 0.2600])


In [2]:
grand_mean = torch.mean(torch.vstack((server_bitflip.data_y, server_bitflip.data_z)),0)

In [10]:
torch.cov(torch.vstack((server_bitflip.data_y, server_bitflip.data_z)).T)

tensor([[ 0.2356, -0.0122, -0.0137, -0.0122],
        [-0.0122,  0.2362, -0.0108, -0.0148],
        [-0.0137, -0.0108,  0.2365, -0.0115],
        [-0.0122, -0.0148, -0.0115,  0.2359]])

In [11]:
(torch.cov(server_bitflip.data_y.T).mul(10000-1) + torch.cov(server_bitflip.data_z.T).mul(10000-1)).div(20000-1)

tensor([[ 0.2356, -0.0122, -0.0137, -0.0122],
        [-0.0122,  0.2361, -0.0108, -0.0148],
        [-0.0137, -0.0108,  0.2365, -0.0114],
        [-0.0122, -0.0148, -0.0114,  0.2359]])

In [18]:
cov = server_bitflip.data_y.sub(grand_mean).T.matmul(server_bitflip.data_y.sub(grand_mean)).add(
    server_bitflip.data_z.sub(grand_mean).T.matmul(server_bitflip.data_z.sub(grand_mean))
).div(20000-1)

In [23]:
torch.linalg.inv(cov).matmul(grand_mean)

tensor([1.9236, 1.9274, 1.9158, 1.9301])

In [6]:
server_bitflip.

[1]

In [ ]:
  def _get_scaling_matrix(self):
        mat_proj = self.get_proj_orth_one_space()
        if self.cuda_device.type== "cpu":
            prec_mat_est =  torch.tensor(numpy.linalg.inv(torch.cov(self.data.T).numpy()))
        else:
            prec_mat_est =  torch.linalg.inv(torch.cov(self.data.T))
        return(
            mat_proj.matmul(prec_mat_est.to(self.cuda_device)).matmul(mat_proj)
        )

In [2]:

p_value_array = np.zeros([n_test, 2])
p_value_array_old = np.zeros([n_test, 2])


for i in range(1):
    t_start_i = time.time()
    torch.manual_seed(i)
    data_priv_y = LDPclient.release_bitflip(
            data_gen.generate_multinomial_data(p1, sample_size),
            alphabet_size,
            privacy_level
            )
    data_priv_z = LDPclient.release_bitflip(
            data_gen.generate_multinomial_data(p2, sample_size),
            alphabet_size,
            privacy_level
    )
    server_bitflip_old.load_private_data_multinomial_y(data_priv_y, alphabet_size)
    server_bitflip_old.load_private_data_multinomial_z(data_priv_z, alphabet_size)

    server_bitflip.load_private_data_multinomial_y(data_priv_y, alphabet_size)
    server_bitflip.load_private_data_multinomial_z(data_priv_z, alphabet_size)

    print("og")
    p_value_array[i,1] = server_bitflip.release_p_value()
    print("perm")
    #p_value_array[i,0] = server_bitflip.release_p_value_permutation(n_permutation)
    
    print("og")
    p_value_array_old[i,1] = server_bitflip_old.release_p_value()
    print("perm")
    #p_value_array_old[i,0] = server_bitflip_old.release_p_value_permutation(n_permutation)
    
    
    t_end_i = time.time() - t_start_i
    print(f"pval: {p_value_array[i,0]} -- {p_value_array[i,1]}(perm), {i+1}th test, time elapsed {t_end_i}")
    print(f"pval old: {p_value_array_old[i,0]} -- {p_value_array_old[i,1]}(perm old), {i+1}th test, time elapsed {t_end_i}\n")




og
tensor([0.3799, 0.3823, 0.3839, 0.3814])


NameError: name 'total_mean' is not defined

In [6]:
torch.mv(
            server_bitflip.get_proj_orth_one_space(),
            server_bitflip.get_mean_y(torch.arange(server_bitflip.n)).sub(
                server_bitflip.get_mean_z(torch.arange(server_bitflip.n))
                )
        )

tensor([ 0.0004, -0.0043,  0.0103, -0.0064])

In [8]:
server_bitflip.get_proj_orth_one_space()

tensor([[ 0.7500, -0.2500, -0.2500, -0.2500],
        [-0.2500,  0.7500, -0.2500, -0.2500],
        [-0.2500, -0.2500,  0.7500, -0.2500],
        [-0.2500, -0.2500, -0.2500,  0.7500]])

In [7]:
server_bitflip.get_mean_y(torch.arange(server_bitflip.n)).sub(
                server_bitflip.get_mean_z(torch.arange(server_bitflip.n))
                )

tensor([-0.0039, -0.0086,  0.0061, -0.0106])